In [80]:
from flask import Flask, render_template, request
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import io
import base64

app = Flask(__name__)

def plot_investment_growth(starting_amount, daily_interests, loss_percent, loss_frequency, days):
    plt.figure(figsize=(10, 5))
    for daily_interest in daily_interests:
        daily_interest = daily_interest / 100.0
        investment_values = np.zeros(days)
        investment_values[0] = starting_amount
        for i in range(1, days):
            if i % loss_frequency == 0:
                investment_values[i] = investment_values[i - 1] * (1 - loss_percent / 100)
            else:
                investment_values[i] = investment_values[i - 1] * (1 + daily_interest)

        plt.plot(investment_values, label=f'{daily_interest * 100:.2f}% daily gain')
    
    plt.title('Investment Increase Over Time')
    plt.xlabel('Days')
    plt.ylabel('Amount ($)')
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x:,.0f}'))
    plt.ylim(0, 1000000)
    plt.legend()
    plt.grid(True)
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    plt.close()
    return plot_url

@app.route('/', methods=['GET', 'POST'])
def home():
    plot_url = None
    if request.method == 'POST':
        starting_amount = float(request.form['starting_amount'])
        loss_percent = float(request.form['loss_percent'])
        loss_frequency = int(request.form['loss_frequency'])
        daily_interests = [float(x) for x in request.form['interest_rates'].split(',')]
        days = int(request.form['days'])
        plot_url = plot_investment_growth(starting_amount, daily_interests, loss_percent, loss_frequency, days)

    return render_template('index.html', plot_url=plot_url)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\josep\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
